In [6]:
!pip install flask
!pip install flask-ngrok
!pip install pyngrok
!pip install markdown2

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os                                                                       #Imports OS to get a way to use operating system
from flask import Flask, request, render_template, redirect, url_for, jsonify   #Imports various functions to work with flask
from werkzeug.utils import secure_filename                                      #Enhances the security of the uploaded file
from pyngrok import ngrok                                                       #Imports the ngrok to manage the tunnel
authtoken = '2oQ7s6Vm4wdy0Mu0Cp7zXYqS27W_4wSAJru6wAmHBY5ebc7w8'                 #Takes the authorized token from the ngrok (User specific)
ngrok.set_auth_token(authtoken)                                                 #Authorizes the ngrok tunnel with the help of the token
import markdown2
import subprocess
import re

# Load the model and tokenizer
model_name = "infly/OpenCoder-1.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Set the computation device
computation = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(computation)

In [16]:
# Initialize Flask app
app = Flask(__name__)

def ensure_code_blocks(response):
    # Fix any improperly formatted code blocks
    response = re.sub(r"(?<!```)```(\w+)?\n", "```\n", response)

    # Wrap code snippets (inside triple backticks) in <pre class='code-block'>
    response = re.sub(r"```(.*?)```", r"<pre class='code-block'>\1</pre>", response, flags=re.DOTALL)

    return response

@app.route('/')
def home():
    return """
    <h1>OpenCoder Model Interface</h1>
    <form action="/generate" method="post">
        <label for="question">Enter your question:</label><br><br>
        <textarea name="question" rows="5" cols="50"></textarea><br><br>
        <input type="submit" value="Generate">
    </form>
    """

@app.route('/generate', methods=['POST'])
def generate():
    # Get the user input
    user_input = request.form['question']

    # Prepare the input for the model
    messages = [{'role': 'user', 'content': user_input}]
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(computation)


    outputs = model.generate(inputs, max_new_tokens=512, do_sample=False)

    # Generate the output
    outputs = model.generate(inputs, max_new_tokens=512, do_sample=False)
    result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    formatted_response = ensure_code_blocks(result)

        # Convert the formatted response to HTML
    final_response = markdown2.markdown(formatted_response)


    # Return the result
    return f"<h1>Model Output:</h1><p>{final_response}</p><br><a href='/'>Go Back</a>"

In [18]:
if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f'Public URL: {public_url}')
    app.run(port=5000)

Public URL: NgrokTunnel: "https://09d2-34-168-57-6.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Nov/2024 10:07:25] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2024 10:07:26] "GET /favicon.ico HTTP/1.1" 404 -
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `at